#### 209 pt 1
##### Question posed:
What factors can be used to predict if a patient will be re-admitted within 30 days?
- Categorical dependent variable

In [0]:
# Set up of notebook

import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import roc_curve, roc_auc_score
from scipy import stats
from scipy.stats.mstats import winsorize
import plotly.figure_factory as ff
# from functions import clean_columns, univariate, winz_outliers # Personal functions

%matplotlib inline

In [0]:
#Import data
med_df = pd.read_csv('medical_clean.csv')
med_df.info()

In [0]:
## Find duplicates
print(med_df.duplicated().value_counts()) # this gives a count of unique values (this case is true/false)
print(med_df.duplicated().sum()) # This gives a count of the nymber of duplicates(counting the true)

In [0]:
#Count number of missing values by columns
med_df.isnull().sum() 

In [0]:
## Create Visualizations of the missing value

plt.figure(figsize=(12, 8))  # Adjust the figure size as necessary

# Create a heatmap to visualize missing data (null values)
# Set cbar=False to hide the color bar and add column names (xticklabels) for clarity
sns.heatmap(med_df.isnull(), cbar=False, cmap='viridis', yticklabels=False, xticklabels=med_df.columns)
plt.xticks(rotation=90)  # Rotate column names for better readability

plt.title('Missing Data Visualization')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

### Rename the data set
I do this on all data sets so that I know I have checked for duplicates and NAs

In [0]:
#Rename dataset
clean_df = med_df.copy()

#Explore the data
clean_df.head()

In [0]:
#Explore summary statistiscs on a data set
clean_df.describe()

In [0]:
# Clean the data set columns using created function based on tresholds
#missing_threshold = 0.95, unique_threshold = 0.95, only 1 value in a column --> removes

#Function for cleaning columns
def clean_columns(df, columns =[], missing_threshold = 0.95, unique_threshold = 0.95, messages = True):
    
    if len(columns) == 0:
        columns = df.columns #this lets the columns be blank and every columns will be cleaned
    
    for col in columns:
        if col in df.columns:
            missing = df[col].isna().sum()
            unique = df[col].nunique()
            rows = df.shape[0]  
            
            if missing / rows >= missing_threshold:
                if messages: print(f"To many missing values with ({missing} out of {rows}, {round((missing / rows) * 100, 2)}%) for {col}, removed")
                df.drop(columns =[col], inplace = True)
            # For non-numeric columns, check if there are too many unique values
            if not pd.api.types.is_numeric_dtype(df[col]) and (unique / rows >= unique_threshold):
                if messages: 
                    print(f"Too many unique values with ({unique} out of {rows}, {round((unique / rows) * 100, 2)}%) for {col}, removed")
                df.drop(columns=[col], inplace=True)
                continue
            elif unique == 1:
                if messages: print(f"Only one value in ({df[col].unique()[0]} for {col}, removed")
                df.drop(columns =[col], inplace = True)
         
        else:
            if messages: print(f"The column variable \"{col}\" doesnt exist as spelled in the DataFrame provided")
     
    return df

clean_columns(clean_df)

In [0]:
#Verify / Re-examine
clean_df.head()

In [0]:
# Remove the index column
clean_df = clean_df.iloc[:, 1:]
#Verify
clean_df.head()

In [0]:
def univariate(df, plot=False):
    stats = []
    
    for col in df.columns:
        col_data = df[col].dropna()  # Drop NA early to avoid repeated calculation
        dtype = col_data.dtype
        desc = col_data.describe() if pd.api.types.is_numeric_dtype(col_data) else None
        
        stats_row = {
            "Variable": col,
            "Type": dtype,
            "Count": desc['count'] if desc is not None else col_data.count(),
            "Missing": df[col].isna().sum(),
            "Unique": col_data.nunique(),
            "Mode": col_data.mode().iloc[0] if not col_data.mode().empty else None
        }

        if pd.api.types.is_numeric_dtype(col_data):
            stats_row.update({
                "Min": desc['min'],
                "Q1": desc['25%'],
                "Median": desc['50%'],
                "Q3": desc['75%'],
                "Max": desc['max'],
                "Mean": desc['mean'],
                "Std": desc['std'],
                "Skew": col_data.skew(),
                "Kurt": col_data.kurt()
            })
            if plot:
                sns.histplot(col_data, kde=True)
                plt.title(f'Histogram of {col}')
                plt.show()
        else:
            stats_row.update({"Min": "-", "Q1": "-", "Median": "-", "Q3": "-", "Max": "-", "Mean": "-", "Std": "-", "Skew": "-", "Kurt": "-"})
            if plot:
                sns.countplot(x=col_data, data=df)
                plt.title(f'Count Plot of {col}')
                plt.show()

        stats.append(stats_row)

    output_df = pd.DataFrame(stats)
    output_df.set_index("Variable", inplace=True)
    return output_df


In [0]:
univariate(clean_df, plot = True) # takes a lot of memory for all plots

In [0]:
## Choose variables to narrow down
# ReAdmis, TotalCharge, Population, Children, Age, Income, VitD_levels, Doc_visits, Full_meals_eaten, vitD_supp, Initial_days, Additional_charges, Gender, Initial_admin, Complication_risk, HighBlood, Diabetes, Hyperlipidemia, Services

knn_data = clean_df[[
    "ReAdmis", "TotalCharge", "Population", "Children", "Age", "Income",
    "VitD_levels", "Doc_visits", "Full_meals_eaten", "vitD_supp",
    "Initial_days", "Additional_charges", "Gender", "Initial_admin",
    "Complication_risk", "HighBlood", "Diabetes", "Hyperlipidemia", "Services"
]].copy()
#Verify new data set
knn_data.head()

In [0]:
## Remove spaces in catagory names within columns
# To replace spaces with underscores in specific columns

for col in knn_data.columns:
     if not pd.api.types.is_numeric_dtype(knn_data[col]):
        knn_data[col] = knn_data[col].str.replace(" ", "_", regex = False) 
        
##Clean White Space and make all lower case
knn_data.columns = knn_data.columns.str.lower().str.strip()       

#round values to 2 decimal points
knn_data = knn_data.round(2)

#Verify space removal/ replacement
knn_data.head()

In [0]:
## Detect and clean outliers

##Function for Winsorization of outliers, using IQR
#https://towardsdatascience.com/detecting-and-treating-outliers-in-python-part-3-dcb54abaf7b0
def winz_outliers(df):
    
    print("Outlier Analysis Report")
    print("=" * 50)  # Print a separator line for visual clarity
    
    for col in df:
        if pd.api.types.is_numeric_dtype(df[col]): 
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))
            outlier_count = outliers.sum()
            
            if outlier_count > 0:
                outer_fence = 3 * IQR
                outer_fence_low = Q1 - outer_fence
                outer_fence_up = Q3 + outer_fence
                
                # Consolidating the print statements for each column
                print(f"\nColumn: {col}")
                print(f"Number of Outliers: {outlier_count}")
                print(f"Lower Outer Fence: {round(outer_fence_low, 2)}")
                print(f"Upper Outer Fence: {round(outer_fence_up, 2)}")

    print("=" * 50)  # End with a separator line for visual clarity


winz_outliers(knn_data)

In [0]:
# Create a boxplot for each column with outliers
outlier_shape = knn_data[["population", "children","income", "vitd_levels", "full_meals_eaten", "vitd_supp", "additional_charges"]]

for column in outlier_shape:
    sns.boxenplot(x=knn_data[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

In [0]:
##choose variable to treat outliers
## Winzorize the outliers
#from scipy.stats.mstats import winsorize
import warnings
## I am Sure that there are no NaN or partition errors
warnings.filterwarnings('ignore', message="Warning: 'partition' will ignore the 'mask' of the MaskedArray")

#Choose variables to winsorize - Population, income, VitD_levels, Full_meals_eaten
#Note that VitD levels will also need a lower limit calculation
#When using the limits, you are specifying "how much" to take off, not the "where" to take off

knn_data['population_winz'] = winsorize(knn_data['population'], limits=(0, 0.05))
knn_data['income_winz'] = winsorize(knn_data['income'], limits=(0, 0.05))
knn_data['vitd_levels_winz'] = winsorize(knn_data['vitd_levels'], limits=(0.05, 0.01))
knn_data['full_meals_winz'] = winsorize(knn_data['full_meals_eaten'], limits=(0, 0.05))

univariate(knn_data)

In [0]:
#Verify
print(knn_data.columns)

In [0]:
#Remake data set with the outliers removed
knn_data = knn_data.drop(columns=["population", "income", "vitd_levels", "full_meals_eaten"], axis = 1)

#verify removal
print(knn_data.columns)

In [0]:
## Function for changing binary values to 0/1 
#AND creating dummy var WITH ALL VARIABLES KEPT

def wrangle_cat(df):
    #handle binary categorical variables
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]):
            # standardize the text format to lowercase
            df[col] = df[col].astype(str).str.strip().str.lower()
            # find binary columns with yes/ no or true/false
            if df[col].isin(['yes', 'no', 'true', 'false']).all():
                mapping = {'yes': 1, 'no': 0, 'true': 1, 'false': 0}
                df[col] = df[col].map(mapping)
    
    # one-hot encode the remaining categorical variables
    # Ensure to only encode those that haven't been converted to numeric in the previous step
    categorical_cols = df.columns[df.dtypes == 'object']
    df = pd.get_dummies(df, columns = categorical_cols, dtype = int, drop_first = False)
                
    return df


knn_encoded = wrangle_cat(knn_data)
print(knn_encoded.columns)

In [0]:
#Verify encoding
knn_encoded.head()

In [0]:
#Lasso for feature selection

X = knn_encoded.drop(["readmis"], axis = 1)
y = knn_encoded["readmis"]

lasso = Lasso(alpha=0.1)
lasso_coef = np.abs(lasso.fit(X, y).coef_)

# Use the columns from X for names
lasso_names = X.columns

# Plot
# https://medium.com/@agrawalsam1997/feature-selection-using-lasso-regression-10f49c973f08

plt.figure(figsize=(10, 6))  # Optional: Adjusts the figure size for better readability
plt.bar(lasso_names, lasso_coef)
plt.xticks(rotation=90)
plt.grid()
plt.title("Feature Selection Based on Lasso")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()

In [0]:
# Feature selection
#Select score_function basedon type of model build wanted.
#https://medium.com/@Kavya2099/optimizing-performance-selectkbest-for-efficient-feature-selection-in-machine-learning-3b635905ed48

skbest = SelectKBest(score_func = f_classif, k ='all')  # Adjust 'k' based on feature selection strategy
X_new = skbest.fit_transform(X, y)
X_new.shape

p_values = pd.DataFrame({"Feature": X.columns, "p_value":skbest.pvalues_}).sort_values("p_value")
p_values[p_values["p_value"] < .05]

In [0]:
#Going to use features from both these models to create the variables for the knn
# "readmis", "totalcharge", "initial_days", "children", "initial_admin_emergency_admission"

knn_features = knn_encoded[["readmis", "totalcharge", "initial_days","services_ct_scan", "services_intravenous", "children", "initial_admin_emergency_admission"]].copy()
knn_features.head()

In [0]:
# DataFrame of knn_encoded has been cleaned and is ready for use in the analysis model.
# Export the data set as csv

# Download clean data as csv
knn_features.to_csv('KNN_features_209.csv', index=False)

In [0]:
# Split the Data
# will split to 80% train, 20% test
# Data will be statitified to ensure accurate proportions of samples from catagories

X = knn_features.drop(["readmis"], axis=1)
y = knn_features["readmis"]

X_train, X_test, y_train, y_test = train_test_split( X, y,
    test_size=0.2,  # Specifies 20% of the data for testing
    random_state=42,  # Ensures reproducibility
    stratify=y  # Stratifies the split based on the labels in 'y'
)

In [0]:
# Standardize the values using scale
scale = StandardScaler()

X_train_scaled = pd.DataFrame(scale.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scale.transform(X_test), columns=X_test.columns)

# Concatenate scaled features with the target variable
train_data = pd.concat([X_train_scaled, y_train.reset_index(drop=True)], axis=1)
test_data = pd.concat([X_test_scaled, y_test.reset_index(drop=True)], axis=1)

# Display the head of the training data
train_data.head()

In [0]:
test_data.head()

In [0]:
#Export testing and training files

X_train_scaled.to_csv("Xscale_train_209.csv", index=False)
X_test_scaled.to_csv("Xscale_test_209.csv", index = False)
y_train.to_csv("Y_train_209.csv", index = False)
y_test.to_csv("Y_test_209.csv", index = False)

In [0]:
# Initialize and train the KNN classifier on the training data
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Evaluate the model on both the training and test datasets
train_accuracy = knn.score(X_train_scaled, y_train)
test_accuracy = knn.score(X_test_scaled, y_test)

print(f'Training Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

In [0]:
##run hyperparameter testing
train_accuracies = {}
test_accuracies = {}
neighbors = np.arange(1, 20)

for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train_scaled, y_train)
    train_accuracies[neighbor] = knn.score(X_train_scaled, y_train)
    test_accuracies[neighbor] = knn.score(X_test_scaled, y_test)
    
    
    
plt.figure(figsize=(8, 6))
plt.title("KNN: Varying Number of Neighbors")
plt.plot(neighbors, train_accuracies.values(), label="Training Accuracy")
plt.plot(neighbors, test_accuracies.values(), label="Testing Accuracy")
plt.legend()
plt.xlabel("Number of Neighbors")
plt.ylabel("Accuracy")
plt.show()

In [0]:
# Find the key (neighbor value) with the highest test accuracy
best_k = max(test_accuracies, key=test_accuracies.get)
best_test_accuracy = test_accuracies[best_k]
best_train_accuracy = train_accuracies[best_k]

print(f"The best value of k is {best_k}, with a test accuracy of {best_test_accuracy:.2f} and a training accuracy of {best_train_accuracy:.2f}")

### Hyperparameter tuning with GridsearchCV 



In [0]:
#Hyperparameter testing

kf = KFold(n_splits = 5,shuffle = True,random_state = 42)
parameter = {'n_neighbors': np.arange(1, 20, 1)}
knn_best = KNeighborsClassifier()
knn_cv = GridSearchCV(knn_best, param_grid = parameter, cv=kf, scoring = "accuracy", verbose = 1)
knn_cv.fit(X_train_scaled, y_train)
print(knn_cv.best_params_)
print(knn_cv.best_score_)

### Final Tuned Model

In [0]:
# Re-run KNN with the best found k value on training data only
knn_best = KNeighborsClassifier(n_neighbors= 7)
knn_best.fit(X_train_scaled, y_train)
y_pred = knn_best.predict(X_test_scaled)

# Evaluate the model again on both the training and test datasets with the best k
train_accuracy_best = knn_best.score(X_train_scaled, y_train)
test_accuracy_best = knn_best.score(X_test_scaled, y_test)

print(f'Best K Training Accuracy: {train_accuracy_best}')
print(f'Best K Test Accuracy: {test_accuracy_best}')

#### Model evaluation

In [0]:
#Determine model accuracy

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

# Creating a DataFrame from the confusion matrix with labels
conf_matrix_df = pd.DataFrame(conf_matrix, 
                              index=['Actual Negative', 'Actual Positive'], 
                              columns=['Predicted Negative', 'Predicted Positive'])

print(conf_matrix_df)

In [0]:
# Normalize by the number of instances in each actual class
conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
sns.heatmap(conf_matrix_norm, annot=True, fmt='.2f', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Normalized Confusion Matrix')
plt.show()


In [0]:
#Interactive confusion matrix

z = conf_matrix
x = ['Predicted No', 'Predicted Yes']
y = ['Actual No', 'Actual Yes']

fig = ff.create_annotated_heatmap(z, x=x, y=y, colorscale='Blues')
fig.update_layout(title='Interactive Confusion Matrix', xaxis=dict(title='Predicted Label'), yaxis=dict(title='Actual Label'))
fig.show()


In [0]:
print(classification_report(y_test, y_pred))

In [0]:
#Sensitivity and specificity

#             Predicted No     Predicted Yes
# Actual No    TN               FP
# Actual Yes   FN               TP

# Precision for the positive class '1'
# precision = TP / (TP + FP)

TP = conf_matrix[1, 1] #True Positives
TN = conf_matrix[0, 0] #True Negatives
FP = conf_matrix[0, 1] #False Positives
FN = conf_matrix[1, 0] #False Negatives

total = (TP + TN + FP + FN)

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
accuracy = (TN + TP) / total
# Precision for the positive class '1'
precision = TP / (TP + FP)

print("Sensitivity (True Positive Rate or Recall):", sensitivity.round(2))
print("Specificity (True Negative Rate):", specificity. round(2))
print("Accuracy: ", accuracy.round(2))
print("Precision for 'Yes' : ", precision.round(2))

In [0]:
## ROC and AUC
# Generate probability scores 
y_pred_prob = knn.predict_proba(X_test_scaled)[ :, 1]

# Calculate ROC curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_prob)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)

# Plotting the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(false_positive_rate, true_positive_rate, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

print(f"AUC: {auc:.4f}")
